## Update blockchain data

In [52]:
import sys
import json
from pathlib import Path
from tqdm.notebook import tqdm
import requests


def find_git_project_root():
    cur_path = Path('.').resolve()
    for i in range(10):
        if (cur_path / '.git').exists():
            return cur_path
        cur_path = cur_path.parent
    raise Exception(f'Unable to find git project root')


def read_json(fpath):
    fpath.parent.mkdir(exist_ok=True, parents=True)

    if not fpath.exists():
        return None

    with fpath.open('r', encoding='utf-8') as f:
        return json.load(f)


def write_json(data, fpath):
    fpath.parent.mkdir(exist_ok=True, parents=True)
    
    # We want to ensure, that if there is an error during saving
    # (for example, not enough memory), there will be some backup file,
    # so that data will not be lost.
    backup_file = None
    if fpath.exists():
        backup_file = fpath.with_name(f'{fpath.stem}.backup.json')
        fpath.rename(backup_file)

    fpath.write_text('', 'utf-8') # we want to make file invalid JSON at this point
    with fpath.open('w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=None, separators=(',', ':'))

    if backup_file:
        backup_file.unlink()
    print('written', fpath.stat().st_size, 'bytes', len(data), 'entries')


def get_operations(sender=None, receiver=None, columns=None):
    """
    Extracts all operations from blockchain by sender or receiver address,
    including internal operations, which are part of batch.
    """

    # https://tzstats.com/docs/api#operations
    max_limit = 50000
    req_res = requests.get('https://api.tzstats.com/tables/op', params={
        'sender': sender,
        'receiver': receiver,
        'limit': max_limit,
        'columns': ','.join(columns),
    }).json()

    if len(req_res) >= max_limit:
        raise Exception(f'Not all transactions were extracted')

    req_res = [
        {
            column: item[column_no]
            for column_no, column in enumerate(columns)
        }
        for item in req_res
    ]

    return req_res


def get_all_transactions_keys():
    res = set()
    columns = ['hash', 'time']

    for addr in tqdm([art_house_contract, nft_contract, comission_wallet]):
        in_ops = get_operations(sender=addr, columns=columns)
        out_ops = get_operations(receiver=addr, columns=columns)
        for op in in_ops + out_ops:
            res.add((op['time'], op['hash']))

    return res


repo_dir = find_git_project_root()
transactions_cache_file = repo_dir / 'cache' / 'transactions.json'
transactions_cache_data = read_json(transactions_cache_file) or {}

art_house_contract = 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9'
nft_contract = 'KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton'
comission_wallet = 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw'

all_transactions_keys = get_all_transactions_keys()

write_json(transactions_cache_data, transactions_cache_file)


written 108440539 bytes 20367 entries


In [68]:
23391227000 / 1000 / 60 / 60 / 24

270.7317939814815

In [67]:
(max([it[0] for it in all_transactions_keys]) - min([it[0] for it in all_transactions_keys]))

23391227000

In [ ]:
transactions_hashes_to_fetch = all_transactions_hashes - set(transactions_cache_data.keys())
for no, batch_hash in enumerate(hashes_to_fetch):
    if batch_hash in all_batch_ops:
        continue
    batch_ops = requests.get(f'https://api.tzstats.com/explorer/op/{batch_hash}').json()
    all_batch_ops[batch_hash] = batch_ops
    print(no, '/', len(hashes_to_fetch), batch_hash)